# Export Hourly
Create multiple GeoJSONs for the interactive map

In [1]:
import geopandas as gpd
import pandas as pd
import json
import consts

In [2]:
# Load Nuts3
gdf_grid_raw = gpd.read_file(consts.PATH_HEXAGON)

# Load timedata
time_data_hourly = json.load(open(consts.PATH_TIMEDATA_HOURLY_HEX, 'r', encoding='utf-8'))
time_data = json.load(open(consts.PATH_TIMEDATA_HEX, 'r', encoding='utf-8'))

In [3]:
GENERATE_SHAPES = [
    # {
    #     'event': 'sunrise',
    #     'time': 'current',
    #     'what': 'latest'
    # },
    {
        'event': 'sunrise',
        'time': 'winter',
        'what': 'latest'
    },
    {
        'event': 'sunrise',
        'time': 'summer',
        'what': 'latest'
    },
    {
        'event': 'sunrise',
        'time': 'winter',
        'what': 'earliest'
    },
    {
        'event': 'sunrise',
        'time': 'summer',
        'what': 'earliest'
    },
    # {
    #     'event': 'sunset',
    #     'time': 'current',
    #     'what': 'earliest'
    # },
    {
        'event': 'sunset',
        'time': 'winter',
        'what': 'earliest'
    },
    {
        'event': 'sunset',
        'time': 'summer',
        'what': 'earliest'
    },
    {
        'event': 'sunset',
        'time': 'winter',
        'what': 'latest'
    },
    {
        'event': 'sunset',
        'time': 'summer',
        'what': 'latest'
    }
]

In [4]:
for action in GENERATE_SHAPES:
    df_timedata = []

    for cell in time_data_hourly:

        record = {
            'nuts_id': cell['nuts_id']
        }

        # Add sunrise current
        for x in cell[action['event']][action['time']][action['what']]:
            record[f"{x['time']}"] = x['elevation']
            # record[f"{x['time']}_day"] = x['day']      

        df_timedata.append(record)

    df_timedata = pd.DataFrame(df_timedata)

    gdf_grid = gdf_grid_raw.merge(df_timedata, left_on='NUTS_ID', right_on='nuts_id')
    gdf_grid = gdf_grid.drop(columns=['nuts_id'])

    # Remove Timezones
    gdf_grid = gdf_grid[~gdf_grid['timezone'].isin(consts.REMOVE_TIMEZONES)]

    # Store
    gdf_grid.to_file(consts.PATH_WEBEXPORT / f"{action['event']}-{action['what']}-{action['time']}.geojson")